In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import pickle

%matplotlib inline

In [89]:
root_path = ''
df_title = pd.read_csv(root_path + 'data/soc-redditHyperlinks-title.tsv', sep='\t')
df_body = pd.read_csv(root_path + 'data/soc-redditHyperlinks-body.tsv', sep='\t')
df_points = pd.read_csv("data/projections/perp_10_2500subs.csv")
df_all = pd.concat([df_title, df_body])

all_subs = set()
all_subs.update(df_points['sub'])

df_all = df_all[df_all.SOURCE_SUBREDDIT.map(lambda x: x in all_subs) | df_all.TARGET_SUBREDDIT.map(lambda x: x in all_subs)]
df_all.TIMESTAMP = pd.to_datetime(df_all.TIMESTAMP)
df_all.sort_values(by='TIMESTAMP', inplace=True)
df_all.reset_index(inplace=True)

df_source = df_all.set_index('SOURCE_SUBREDDIT')
df_target = df_all.set_index('TARGET_SUBREDDIT')

In [3]:
def filter_dates(df, start, end):
    return 

def filter_subs():
    return

Linked To:
Linked From:

Net Received Sentiment
Net Sent Sentiment

Daily To Net Sentiment:
Daily Out Net Sentiment:

In [ ]:
def link_counts(df):
    mask = df.LINK_SENTIMENT == 1
    positive = df[mask]
    negative = df[~mask]
    
    return [positive.index.value_counts(), negative.index.value_counts()]

In [ ]:
# # def get_link_counts()
# def process_frame(df_points):
#     pcounts, ncounts = link_counts(df_target)
#     df_points['in_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
#     df_points['in_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))

#     pcounts, ncounts = link_counts(df_source)
#     df_points['out_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
#     df_points['out_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))
#     return df_points

In [ ]:
# def get_link_counts()
def process_frame(df_points):
    pcounts, ncounts = link_counts(df_target)
    df_points['in_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
    df_points['in_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))
    df_points['in_links'] = df_points['in_neg'] + df_points['in_pos']

    pcounts, ncounts = link_counts(df_source)
    df_points['out_pos'] = df_points['sub'].map(lambda x: pcounts.get(x, 0))
    df_points['out_neg'] = df_points['sub'].map(lambda x: ncounts.get(x, 0))
    df_points['out_links'] = df_points['out_pos'] + df_points['out_neg']
    return df_points

In [ ]:
import multiprocessing as mp
cores = mp.cpu_count()
pool = mp.Pool(cores)
splits = np.array_split(df_points, cores)

f = pool.map(process_frame,splits)
pool.close()
pool.join()

In [119]:
bins = pd.cut(df_all.TIMESTAMP, 50)
df_all['bin'] = bins
pos_rows = df_all[df_all.LINK_SENTIMENT == 1]
neg_rows = df_all[df_all.LINK_SENTIMENT == -1]
df_vol = pd.DataFrame(pos_rows.groupby('bin')['POST_ID'].count().values, columns=['positive'])
df_vol['negative'] = neg_rows.groupby('bin')['POST_ID'].count().values

In [99]:
filtered_pos = bins.loc[df_all[df_all.LINK_SENTIMENT == 1].index]

In [103]:
%timeit df_all[df_all.LINK_SENTIMENT == 1]

121 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
%timeit df_all.loc[pos_rows]

170 ms ± 6.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [83]:
df_test = df_all.copy()

In [94]:
df_all.loc[23]

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
23,agitation,changemyview,1u5cj7s,2013-12-31 16:20:20,1,"102.0,91.0,0.843137254902,0.0,0.0882352941176,..."
23,corejerk,bestof2013,1u5w42s,2014-01-01 04:58:04,1,"2735.0,2451.0,0.722120658135,0.0197440585009,0..."


In [110]:
popos_rows.groupby('bin').count()['index'].values

(50,)

In [115]:
df_vol

,positive,negative
0,8974,965
1,9829,1108
2,10269,1153
3,10241,1208
4,10533,1276
5,11030,1262
6,10790,1263
7,10356,1224
8,11860,1325
9,11769,1298
